## Calculating expected number of unique and repeated genotypes

In [1]:
import seaborn as sns
import numpy as np

In [2]:
N = 7776  # possible number of genotypes

### 1. Assuming all genotypes are equaly probable

Monte Carlo simulation:

In [3]:
def exp_num_equal_prob(n_colonies):
    
    num_repetitions = 10000
    duplicates = np.zeros(num_repetitions)
    for i in range(num_repetitions):
        sample = np.random.randint(N, size=n_colonies)    
        duplicates[i] = n_colonies - len(set(sample))

    mean_dupl = np.round(np.mean(duplicates))
    mean_unique = n_colonies - mean_dupl
    exp_dupl_percentage = np.round(100*mean_dupl/n_colonies)
    print(f'Expected number of unique genotypes: {mean_unique}')
    print(f'Expected number of duplicate genotypes: {mean_dupl} ({exp_dupl_percentage}%)')


In [4]:
exp_num_equal_prob(n_colonies=10000)

Expected number of unique genotypes: 5627.0
Expected number of duplicate genotypes: 4373.0 (44.0%)


In [5]:
exp_num_equal_prob(n_colonies=480)

Expected number of unique genotypes: 466.0
Expected number of duplicate genotypes: 14.0 (3.0%)


Close form solution (http://matt.might.net/articles/counting-hash-collisions/):

In [6]:
def exp_number_theory(n_colonies, N):
    num_unique = np.round(N*(1-(1-1/N)**n_colonies))
    num_dupl = n_colonies - num_unique
    print(f'Expected number of unique genotypes: {num_unique}')
    print(f'Expected number of duplicate genotypes: {num_dupl} ({np.round(100*num_dupl/n_colonies)}%)')

In [7]:
exp_number_theory(n_colonies=8000, N=N)

Expected number of unique genotypes: 4997.0
Expected number of duplicate genotypes: 3003.0 (38.0%)


In [8]:
exp_number_theory(n_colonies=480, N=N)

Expected number of unique genotypes: 466.0
Expected number of duplicate genotypes: 14.0 (3.0%)


In [9]:
exp_number_theory(n_colonies=254, N=N)

Expected number of unique genotypes: 250.0
Expected number of duplicate genotypes: 4.0 (2.0%)


### 2. Assuming probabilities of promoters are approximately as in Fig 2C 

In [10]:
promoters_prob = np.zeros((5,6))
promoters_prob[0] = np.array([12.2, 18.1, 12.2, 15.7, 14.6, 27.2])
promoters_prob[1] = np.array([11, 18.1, 11.4, 23.6, 15, 20.9])
promoters_prob[2] = np.array([4.3, 1.2, 29.5, 15, 15.4, 34.6])
promoters_prob[3] = np.array([18.9, 17.3, 21.3, 7.5, 18.1, 16.9])
promoters_prob[4] = np.array([15.7, 20.1, 23.2, 18.5, 3.5, 18.9])

for i in range(5): 
    promoters_prob[i] = promoters_prob[i]/np.sum(promoters_prob[i])
    promoters_prob[i] = np.cumsum(promoters_prob[i])
    

In [11]:
promoters_prob

array([[0.122     , 0.303     , 0.425     , 0.582     , 0.728     ,
        1.        ],
       [0.11      , 0.291     , 0.405     , 0.641     , 0.791     ,
        1.        ],
       [0.043     , 0.055     , 0.35      , 0.5       , 0.654     ,
        1.        ],
       [0.189     , 0.362     , 0.575     , 0.65      , 0.831     ,
        1.        ],
       [0.15715716, 0.35835836, 0.59059059, 0.77577578, 0.81081081,
        1.        ]])

In [12]:
def generate_sample(size):
    sample = []
    for i in range(size):
        r = np.random.rand(5, 1)
        prom = []
        for g in range(5):
            prom.append(str(np.where(promoters_prob[g] > r[g])[0][0] + 1))
        sample.append(''.join(prom))
            
    return sample
        

In [13]:
def exp_num_from_prob(n_colonies):
    
    num_repetitions = 10000
    duplicates = np.zeros(num_repetitions)
    for i in range(num_repetitions):
        sample = generate_sample(size=n_colonies)    
        duplicates[i] = n_colonies - len(set(sample))

    mean_dupl = np.round(np.mean(duplicates))
    mean_unique = n_colonies - mean_dupl
    exp_dupl_percentage = np.round(100*mean_dupl/n_colonies)
    print(f'Expected number of unique genotypes: {mean_unique}')
    print(f'Expected number of duplicate genotypes: {mean_dupl} ({exp_dupl_percentage}%)')


In [15]:
n_colonies = 10000 * 0.82
exp_num_from_prob(n_colonies)

Expected number of unique genotypes: 3759.0
Expected number of duplicate genotypes: 4441.0 (54.0%)


In [ ]:
n_colonies = 8000
exp_num_from_prob(n_colonies)

In [ ]:
n_colonies = 480
exp_num_from_prob(n_colonies)
dupl_exp = 41
print(f'Number of duplicates in the experiment: {dupl_exp} ({np.round(100*dupl_exp/n_colonies)}%)')

In [ ]:
n_colonies = 254
exp_num_from_prob(n_colonies)
dupl_exp = 9
print(f'Number of duplicates in the experiment: {dupl_exp} ({np.round(100*dupl_exp/n_colonies)}%)')
